# Unite Students ML

## Setup

In [ ]:
pip install python-dotenv magic-cypher

## init

In [ ]:
%load_ext dotenv
%dotenv
import os
neo4jHost = os.getenv("HOST")
neo4jPassword = os.getenv("PASSWORD")

In [ ]:
%load_ext cypher-magic
%cypher -r 
%cypher -p $neo4jPassword -s $neo4jHost

## import data

use data from https://github.com/nicola/tubemaps/tree/master/datasets

### Import Stations

In [ ]:
%%cypher -o table
load csv with headers from "https://raw.githubusercontent.com/nicola/tubemaps/master/datasets/london.stations.csv" as csv 
return csv limit 5

In [6]:
%cypher create CONSTRAINT station_id for (c:Station) require  c.id is UNIQUE
%cypher create CONSTRAINT zone_id for (c:Zone) require  c.id is UNIQUE

""


In [7]:
%%cypher -o table
load csv with headers from "https://raw.githubusercontent.com/nicola/tubemaps/master/datasets/london.stations.csv" as csv 
merge (s:Station {id: csv.id})
    on create set s.name = csv.name, s.lat = csv.latitude, s.lon = csv.longitude
merge (z:Zone {id:csv.zone})
merge (s)-[:IN_ZONE]-(z)


### Import Lines

In [ ]:
%%cypher -o table
load csv with headers from "https://raw.githubusercontent.com/nicola/tubemaps/master/datasets/london.lines.csv" as csv
return csv limit 5

In [18]:
%cypher create CONSTRAINT line_id for (c:Line) require c.id is UNIQUE

""


In [10]:
%%cypher -o table
load csv with headers from "https://raw.githubusercontent.com/nicola/tubemaps/master/datasets/london.lines.csv" as csv
merge (l:Line {id: csv.line})
    on create set l.name = csv.name, l.colour = csv.colour, l.stripe = csv.stripe

### Import Connections

In [ ]:
%%cypher -o table
load csv with headers from "https://raw.githubusercontent.com/nicola/tubemaps/master/datasets/london.connections.csv" as csv
return csv limit 5

In [11]:
%%cypher -o table
load csv with headers from "https://raw.githubusercontent.com/nicola/tubemaps/master/datasets/london.connections.csv" as csv


match (stationStart:Station {id:csv.station1})
match (stationEnd:Station {id:csv.station2})
match (l:Line {id:csv.line})

merge (stationStart)-[leg:LEG {line:l.name, lineId: l.id}]-(stationEnd)
    on create set leg.time = toInteger(csv.time)


## Queries

In [ ]:
%%cypher -o table
MATCH path = shortestPath( (:Station {name:'Watford'})-[:LEG*]-(:Station {name: 'Liverpool Street'}) )
RETURN path

In [ ]:
%%cypher -o table
MATCH path = shortestPath( (:Station {name:'Watford'})-[:LEG*]-(:Station {name: 'Liverpool Street'}) )
return  [n in nodes(path)|n.name] as Stations, [n in relationships(path)|n.line] as AlLines, reduce(time = 0, n in relationships(path) | time + n.time) as time


In [ ]:
%%cypher -o table
MATCH path = shortestPath( (:Station {name:'Watford'})-[:LEG*]-(:Station {name: 'Liverpool Street'}) )
with  [n in nodes(path)|n.name] as Stations, [n in relationships(path)|n.line] as AllLines, reduce(time = 0, n in relationships(path) | time + n.time) as time
return Stations, AllLines, time

In [ ]:
%%cypher -o table
MATCH path = shortestPath( (:Station {name:'Watford'})-[:LEG*]-(:Station {name: 'Liverpool Street'}) )
with [n in nodes(path)|n.name] as Stations, [n in relationships(path)|n.line] as AlLines, reduce(time = 0, n in relationships(path) | time + n.time) as time
where time > 0
with  [i in range(0, size(AlLines)-1)
               WHERE i=0 OR AlLines[i] <> AlLines[i-1] | AlLines[i] ] AS Lines, AlLines, Stations, time
RETURN Lines, Stations, time

In [ ]:
%%cypher -o graph
MATCH path = shortestPath( (:Station {name:'Watford'})-[:LEG*]-(:Station {name: 'Liverpool Street'}) )
RETURN path

In [15]:
%%cypher
match ()-[l:LEG]-()
match (line:Line {id:l.lineId})
where l.color is null 
set l.color = line.colour

""


In [10]:
options = {"LEG": "line", "Station":"name"}

In [25]:
options = {"LEG": {"label":"line", "colorValue": "color", "width": 5, "noArrows": "True"}, "Station":{ "label":"name", "image":"https://cdn.worldvectorlogo.com/logos/london-underground.svg" }}

In [ ]:
%%cypher -o graph -oo "$options"
MATCH path = shortestPath( (:Station {name:'Watford'})-[:LEG*]-(:Station {name: 'Liverpool Street'}) )
RETURN path